# Mentoría DiploDatos 2025 - Segmentación Inteligente de Clientes en Telecomunicaciones


### Introduccion

El proyecto consiste en desarrollar un modelo de segmentación de clientes para una empresa del sector de telecomunicaciones, con el objetivo de identificar patrones de comportamiento y definir estrategias de acción personalizadas para cada grupo, generando así valor comercial y fortaleciendo la toma de decisiones de negocio.

Este proyecto pone en evidencia el valor de la clusterización y el uso de modelos no supervisados como herramientas clave para conocer en profundidad las características de los clientes. A partir de los segmentos identificados, es posible proyectar acciones concretas orientadas a mejorar la fidelización, impulsar ventas cruzadas y desarrollar estrategias efectivas de retención.

Es fundamental recordar que no todos los clientes son iguales, y que al aprovechar los datos de manera inteligente, podemos ofrecerles una experiencia más personalizada, relevante y satisfactoria.

### Inicializacion del entorno

In [1]:
!pip install pandas 


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install plotly


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install matplotlib


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install seaborn


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install scipy


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install sqlalchemy


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install cx_Oracle


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# Importación de librerías
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

import itertools
import warnings
warnings.filterwarnings("ignore")
import io

# Para visualización
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot
from plotly.figure_factory import create_table
init_notebook_mode(connected=True)

import matplotlib.pyplot as plt
import seaborn as sns

# Librerías de aprendizaje automático
from sklearn.cluster import KMeans, MeanShift, DBSCAN 
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.inspection import permutation_importance

# Librerías estadísticas y matemáticas
from scipy.stats import t
from scipy import stats
from math import pi

In [10]:
#Carga de datos
df = pd.read_csv("Segmentacion_Clientes.csv", sep=";")
print(df.head())

                  RAZON_SOCIAL  REGION_OTROS  REGION_CF  REGION_BSAS  \
0              AVANZ TRUCK SRL             1          0            0   
1             COOLMEALS S.A.S.             0          0            0   
2             GRUPO QUEST S.A.             0          0            0   
3  3NTROPY TECHNOLOGIES S.R.L.             0          1            0   
4                    48M2 S.A.             0          1            0   

   REGION_DESCONOCIDO  REGION_CBA  PROD_DATOS_RPV  PROD_INTER_DEDICADO  \
0                   0           0               0                    1   
1                   0           1               0                    1   
2                   0           1               1                    1   
3                   0           0               0                    1   
4                   0           0               0                    1   

   PROD_DATOS_ENLACE  PROD_DATOS_LAN_TO_LAN  PROD_SD_WAN  MERC_OTROS  \
0                  0                      0       

### Detalle del dataset por feature:
- RAZON_SOCIAL: Denominación del Cliente Corporativo
- REGION_CF: Si el cliente está ubicado en Capital Federal es un 1, caso contrario 0.
- REGION_BSAS: Si el cliente está ubicado en Bs. As es un 1, caso contrario 0.
- REGION_CBA: Si el cliente está ubicado en Córdoba es un 1, caso contrario 0.
- REGION_DESCONOCIDO: Clientes cuyo campo de Ubicación está vacío. Se arma esta variable para que no haya nulos.
- REGION_OTROS: Incluye las localidades que no están en las demas regiones (por ejemplo clientes localizados en Misiones tendrán un 1 en este campo)
- PROD_DATOS_RPV: Clientes que tienen contratado el servicio de RPV(Red Privada Virtual). Esta es una solución de telecomunicaciones que permite interconectar distintas sedes de una empresa a través de una red privada y segura, utilizando la infraestructura de un proveedor (como una operadora de telecomunicaciones).
- PROD_INTER_DEDICADO: Clientes que tienen contratado el servicio de Internet dedicado.Este consiste en en brindar una conexión exclusiva y simétrica a internet con ancho de banda garantizado para una empresa. 
- PROD_DATOS_ENLACE: Clientes que tienen contratado el servicio de enlace.Enlace es una conexión punto a punto que une físicamente dos ubicaciones para transmitir datos directamente.
- PROD_DATOS_LAN_TO_LAN:Clientes que tienen contratado el servicio de lan to lan. Esto permite interconectar dos o más redes locales (LAN) de distintas sedes mediante enlaces privados y seguros.
- PROD_SD_WAN:Clientes que tienen contratado el servicio de SD WAN. Esta es una tecnología que gestiona de forma inteligente múltiples conexiones de red (como MPLS, internet o 4G) para optimizar el rendimiento y la seguridad del tráfico entre sedes.
- MERC_CORPORATIVO: Indica con 1 si el cliente es del segmento corporativo y 0 caso contrario.
- MERC_EMPRESAS: Indica con 1 si el cliente es del segmento Empresas y 0 caso contrario.
- MERC_GRANDES_CLIENTES: Indica con 1 si el cliente es del segmento Grandes Clientes y 0 caso contrario.
- MERC_PYMES: Indica con 1 si el cliente es del segmento Pymes y 0 caso contrario.
- MERC_OTROS: Indica con 1 si el cliente es de un segmento diferente a los anteriores y 0 caso contrario.
- CANT_PRODUCTOS: Contabiliza la cantidad de productos de cada cliente.
- CANT_ENLACES: Contabiliza la cantidad de enlaces de cada cliente. Los clientes pueden tener mas de un enlace y más de un producto.
- FACTURACION_ARS: Indica lo que paga en pesos el cliente de acuerdo a la cantidad de enlaces
- DURACION_CLIENTE_DIAS: Indica la cantidad de días transcurridos desde que el cliente adquirió un producto. 
- INCIDENTES: Contabiliza la cantidad de reclamos del cliente debido a incidentes en un enlace.
- TMR: El Tiempo medio de respuesta indica cuantas horas pasaron desde que se reportó el incidente hasta que se resuelve.

#### **Trabajo practico 2**: Analisis Exploratorio y Curación de datos
En esta etapa acondicionamos el dataset para alimentar el modelo:

##### Cuestiones a analizar y responder:
1. **Valores faltantes (Nan, NA o None)**: ¿Existen valores faltantes en el conjunto de datos? ¿Porqué piensan que ocurrió ésto?¿Deberíamos eliminar las filas/columnas o imputar valores?

2. **Outliers**: ¿Existen valores atipicos? ¿Que tecnica utilizarán para la deteccion y manejo de estos valores?

3. **Tratamiento de features**: Divida las variables en numericas y categoricas. ¿Es necesario codificar variables categoricas en numericas?

4. **Escalado y Normalización**: ¿Necesitamos escalar o normalizar las variables numéricas? ¿Qué técnicas de escalado o normalización deberíamos usar?

5. **Feature engineering**:¿Existe relacion entre las variables?¿Se elimina alguna variable? ¿Creen necesario reducir las variables a través de PCA?Justificar las ventajas y desventajas de aplicarlo

In [13]:
#Considerar solo las columnas numéricas para escalar
columnas = np.r_[1:22]  # Esto va desde el índice 1 hasta el 21 inclusive
df_numeric = df.iloc[:, columnas]

In [15]:
# Reemplazar ',' por '.' y convertir a float
df_numeric = df_numeric.apply(lambda col: col.str.replace(',', '.', regex=False) if col.dtypes == 'object' else col)
df_numeric = df_numeric.apply(pd.to_numeric, errors='coerce')

In [16]:
# Estandarización de los datos
std_scaler = StandardScaler()
standard = std_scaler.fit_transform(df_numeric)
df_escalado=pd.DataFrame(standard,columns=df_numeric.columns)

In [17]:
pca = PCA()
pca.fit(df_escalado)
print("Varianza explicada por cada componente (estandarizado):", pca.explained_variance_ratio_.cumsum())

Varianza explicada por cada componente (estandarizado): [0.16386484 0.25245005 0.3250146  0.39388676 0.46078897 0.52178873
 0.58258094 0.63665914 0.68828419 0.7378013  0.78515139 0.82986205
 0.87061456 0.90785942 0.94315005 0.97229053 0.99074131 0.99973325
 0.99996226 1.         1.        ]


In [18]:
pca=PCA(n_components=13)
# Transformar los datos y obtener las componentes principales
projected_features_std = pca.fit_transform(df_escalado)

# Crear un DataFrame con las dos primeras componentes principales
df_escalado_pca = pd.DataFrame(projected_features_std)

### Conclusiones y Observaciones